In [1]:
%matplotlib inline

import sys
import skimage.exposure
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython.display import Image
import datacube
from odc.ui import with_ui_cbk
from pyproj import Proj, transform
from datacube import Datacube
from datacube.utils import masking
import math
from osgeo import gdal
from datacube.model import Range
from datetime import datetime
import numpy as np
import os, json, requests, time





ARDremotedc = Datacube(config='/g/data/up71/projects/ARD_board_paper_Data/access_examples/ard_interoperability.conf')


In [2]:
products = ARDremotedc.list_products()

display_columns = ["name",
                   "description",
                   "platform",
                   "instrument",
                   "crs",
                   "resolution"]

products[display_columns].sort_index()

,name,description,platform,instrument,crs,resolution
id,,,,,,
1,ls8_ard,Landsat 8 ARD datasets for the GA/USGS interop...,LANDSAT_8,OLI,NaN,NaN
2,ls7_ard,Landsat 7 ARD datasets for the GA/USGS interop...,LANDSAT_7,ETM+,NaN,NaN
3,ls5_ard,Landsat 5 ARD datasets for the GA/USGS interop...,LANDSAT_5,TM,NaN,NaN
4,ls5_usgs_l2c1,Landsat 5 Thematic Mapper (TM) USGS Analysis R...,LANDSAT_5,TM,NaN,NaN
5,ls7_usgs_l2c1,Landsat 7 Enhanced Thematic Mapper Plus (ETM+)...,LANDSAT_7,ETM,NaN,NaN
6,ls8_usgs_l2c1,Landsat 8 Operational Land Imager (OLI) and Th...,LANDSAT_8,OLI_TIRS,NaN,NaN
7,ls8_level1_usgs,Landsat 8 USGS Level 1 Collection-1 OLI-TIRS,LANDSAT_8,OLI_TIRS,NaN,NaN
8,ls7_level1_usgs,Landsat 7 USGS Level 1 Collection-1 OLI-TIRS,LANDSAT_7,ETM,NaN,NaN
9,ls5_level1_usgs,Landsat 5 USGS Level 1 Collection-1 OLI-TIRS,LANDSAT_5,TM,NaN,NaN


In [3]:
product = "usgs_ls8c_level2_2"
measurements = ARDremotedc.list_measurements()
measurements.loc[product]

,name,dtype,units,nodata,aliases,flags_definition,spectral_definition
measurement,,,,,,,
coastal_aerosol,coastal_aerosol,uint16,1,0,[coastal_aerosol],NaN,NaN
blue,blue,uint16,1,0,[blue],NaN,NaN
green,green,uint16,1,0,[green],NaN,NaN
lwir,lwir,uint16,1,0,[lwir],NaN,NaN
nir,nir,uint16,1,0,[nir],NaN,NaN
red,red,uint16,1,0,[red],NaN,NaN
swir_1,swir_1,uint16,1,0,[swir_1],NaN,NaN
swir_2,swir_2,uint16,1,0,[swir_2],NaN,NaN
qa_aerosol,qa_aerosol,uint8,1,1,[qa_aerosol],NaN,NaN


In [4]:
query = {
  'time': ('2018-12-01', '2019-01-01'),
  'lat': (-30.585, -30.586),
  'lon': (115.156, 115.157),
  'output_crs' : 'EPSG:3577',
  'resolution': (-30, 30), 
}

USGS_ds = ARDremotedc.load(product='usgs_ls8c_level2_2', **query)

In [5]:
USGS_ds.swir_2[0]

<xarray.DataArray 'swir_2' (y: 5, x: 4)>
array([[26741, 26301, 25981, 25978],
       [26181, 26420, 26055, 25999],
       [25933, 26220, 26681, 26544],
       [26059, 26241, 26690, 26576],
       [26017, 26207, 26649, 26696]], dtype=uint16)
Coordinates:
    time         datetime64[ns] 2018-12-14T02:11:06.781432
  * y            (y) float64 -3.434e+06 -3.434e+06 ... -3.434e+06 -3.434e+06
  * x            (x) float64 -1.594e+06 -1.594e+06 -1.594e+06 -1.594e+06
    spatial_ref  int32 3577
Attributes:
    units:         1
    nodata:        0
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

In [6]:
query = {
  'time': ('2020-01-01', '2020-01-21'),
  'lat': (-35.093052, -35.092252),
  'lon': (149.46134, 149.462778),
  'output_crs' : 'EPSG:3577',
  'resolution': (-10, 10), 
}
ESA_ds = ARDremotedc.load(product='s2a_sen2cor_v1', **query)

In [7]:
ESA_ds

<xarray.Dataset>
Dimensions:      (time: 2, x: 15, y: 12)
Coordinates:
  * time         (time) datetime64[ns] 2020-01-10T00:04:26.666010 2020-01-20T...
  * y            (y) float64 -3.941e+06 -3.941e+06 ... -3.941e+06 -3.941e+06
  * x            (x) float64 1.583e+06 1.583e+06 ... 1.583e+06 1.583e+06
    spatial_ref  int32 3577
Data variables:
    B02_10m      (time, y, x) uint16 1164 1158 1134 1098 ... 15528 16368 16232
    B03_10m      (time, y, x) uint16 1324 1310 1274 1268 ... 14304 15584 15536
    B04_10m      (time, y, x) uint16 1534 1488 1456 1438 ... 13528 14616 15232
    B08_10m      (time, y, x) uint16 1830 1802 1726 1722 ... 13984 15048 14968
    AOT_10m      (time, y, x) uint16 184 184 184 184 184 ... 184 184 184 184 184
    WVP_10m      (time, y, x) uint16 1807 1807 1745 1745 ... 1913 1913 1913 1913
    B05_20m      (time, y, x) uint16 1613 1613 1542 1542 ... 13869 15620 15620
    B06_20m      (time, y, x) uint16 1633 1633 1558 1558 ... 13008 14522 14522
    B07_20m      (time, y, x) uint16 1734 1734 1653 1653 ... 12605 13780 13780
    B11_20m      (time, y, x) uint16 3011 3011 2947 2947 ... 7759 7759 8264 8264
    B12_20m      (time, y, x) uint16 2705 2705 2629 2629 ... 6110 6110 6297 6297
    B8A_20m      (time, y, x) uint16 1880 1880 1775 1775 ... 12234 13376 13376
    AOT_20m      (time, y, x) uint16 184 184 184 184 184 ... 184 184 184 184 184
    WVP_20m      (time, y, x) uint16 1807 1807 1745 1745 ... 1913 1913 1913 1913
    SCL_20m      (time, y, x) uint16 5 5 5 5 5 5 5 5 5 5 ... 9 9 9 9 9 9 9 9 9 9
    B01_60m      (time, y, x) uint16 929 929 958 958 ... 14105 14105 16162 16162
    B09_60m      (time, y, x) uint16 1733 1733 1781 1781 ... 15970 15962 15962
    AOT_60m      (time, y, x) uint16 184 184 184 184 184 ... 184 184 184 184 184
    WVP_60m      (time, y, x) uint16 1716 1716 1712 1712 ... 1913 1913 1913 1913
    SCL_60m      (time, y, x) uint16 5 5 5 5 5 5 5 5 5 5 ... 9 9 9 9 9 9 9 9 9 9
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

In [8]:
ESA_ds['time']

<xarray.DataArray 'time' (time: 2)>
array(['2020-01-10T00:04:26.666010000', '2020-01-20T00:04:26.333979000'],
      dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-01-10T00:04:26.666010 2020-01-20T...
    spatial_ref  int32 3577
Attributes:
    units:    seconds since 1970-01-01 00:00:00

In [10]:
import rasterio
rasterio.__version__

'1.1.4'